In [1]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'mejiro':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from glob import glob
import multiprocessing
import pandas as pd
from tqdm import tqdm
import time
from copy import deepcopy
from multiprocessing import Pool
from hydra import initialize, compose
import pickle

from mejiro.helpers.test_physical_lens import TestPhysicalLens
from mejiro.helpers.lens import Lens
from mejiro.plots import diagnostic_plot, plot
from mejiro.utils import util
from mejiro.scripts import generate
from mejiro.helpers import pyhalo


In [3]:
def add_subhalos(lens):
    # add CDM subhalos
    try:
        lens.add_subhalos(*pyhalo.generate_CDM_halos(lens.z_lens, lens.z_source))
    except:
        pass

In [4]:
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, repo_dir, pickle_dir = config.machine.array_dir, config.machine.data_dir, config.machine.repo_dir, config.machine.pickle_dir
util.create_directory_if_not_exists(pickle_dir)

In [5]:
# open pickled lens list
with open(os.path.join(pickle_dir, 'skypy_output_lens_list'), 'rb') as results_file:
    lens_list = pickle.load(results_file)

In [6]:
# split up the lenses into batches based on core count
cpu_count = multiprocessing.cpu_count()
process_count = int(cpu_count / 2)  # TODO consider making this larger, but using all CPUs has crashed
generator = util.batch_list(lens_list, process_count)
batches = list(generator)

In [7]:
# process the batches
for batch in tqdm(batches):
    pool = Pool(processes=process_count) 
    for i, output in enumerate(pool.map(add_subhalos, batch)):
        (lens) = output
        lens_list.append(lens)

  0%|          | 0/11588 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# pickle lens list with subhalos
with open(os.path.join(pickle_dir, 'skypy_output_lens_list_with_subhalos'), 'ab') as results_file:
    pickle.dump(lens_list, results_file)